In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
from matplotlib import pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier

In [2]:
d = open('./student-por.csv','r')
data_lst = [line.strip('\n').replace('"','').split(';') for line in d.readlines()]
data = pd.DataFrame(data_lst, columns = data_lst[0])
pd.set_option('display.max_columns', None)
data = data.drop(index = 0)
data = data.reset_index()
data = data.apply(pd.to_numeric,errors = 'ignore')
data.set_index(["index"], inplace=True)
data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,4,0,11,11
2,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,2,9,11,11
3,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,0,yes,no,no,no,yes,yes,yes,no,4,3,2,2,3,3,6,12,13,12
4,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,no,yes,yes,yes,yes,yes,3,2,2,1,1,5,0,14,14,14
5,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,no,no,yes,yes,no,no,4,3,2,1,2,5,0,11,13,13


In [3]:
list(data['G1']).count(0)

1

In [4]:
grade = []
for i in data.G1:
    if i <= 9:
        grade.append('A')
    elif i>9 and i<=15:
        grade.append('B')
    elif i>15:
        grade.append('C')
    else:
        grade.append('error')
data['g1']=grade
data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,g1
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,4,0,11,11,A
2,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,2,9,11,11,A
3,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,0,yes,no,no,no,yes,yes,yes,no,4,3,2,2,3,3,6,12,13,12,B
4,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,no,yes,yes,yes,yes,yes,3,2,2,1,1,5,0,14,14,14,B
5,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,no,no,yes,yes,no,no,4,3,2,1,2,5,0,11,13,13,B


In [5]:
data['g1'].value_counts()

B    446
A    157
C     46
Name: g1, dtype: int64

In [6]:
data['g1'].value_counts('B')

B    0.687211
A    0.241911
C    0.070878
Name: g1, dtype: float64

In [7]:
demographic_data_lst = ['school','sex','age','address','famsize','Pstatus','Medu','Fedu','Mjob','Fjob','reason','guardian','nursery','internet','famsup']
data_to_onehot = ['school','sex','address','famsize','Pstatus','Mjob','Fjob','reason','guardian','nursery','internet','famsup']

In [8]:
data_demo = data[demographic_data_lst]

In [9]:
data2=pd.get_dummies(data_demo)

In [10]:
train_data_header = data2.columns.tolist()
train_data = data2.values
train_data.shape

(649, 36)

In [11]:
train_x = train_data[:, 0:-3]
train_y = data.values[:, -1]

In [12]:
train_x.shape

(649, 33)

### MultinomialNB

In [13]:
model = MultinomialNB()
model

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [14]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(train_x,train_y,test_size=0.3,random_state=123)

In [15]:
model.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [16]:
pred = model.predict(x_test)

In [17]:
accuracy_score(y_test,pred)

0.6820512820512821

In [18]:
for i in ['A','B','C']:
    print(i,':',list(pred).count(i))

A : 46
B : 147
C : 2


In [19]:
for i in ['A','B','C']:
    print(i,':',list(y_test).count(i))

A : 35
B : 142
C : 18


In [20]:
y_test

array(['B', 'B', 'A', 'B', 'B', 'B', 'A', 'A', 'B', 'B', 'B', 'B', 'B',
       'C', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'C', 'B', 'B', 'B', 'A',
       'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'C', 'B', 'A', 'B', 'B', 'A', 'B', 'B', 'B', 'B', 'A', 'A',
       'B', 'A', 'A', 'B', 'A', 'B', 'A', 'B', 'A', 'B', 'B', 'A', 'B',
       'A', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'B',
       'C', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'C', 'B', 'B',
       'A', 'C', 'A', 'A', 'B', 'B', 'C', 'C', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'C', 'B', 'B', 'B', 'A', 'B', 'C', 'B', 'B',
       'B', 'B', 'B', 'B', 'A', 'A', 'B', 'A', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'A', 'A', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'C', 'A', 'C', 'B', 'B', 'B', 'B', 'B',
       'B', 'C', 'B', 'B', 'A', 'B', 'B', 'B', 'B', 'C', 'B', 'B', 'B',
       'C', 'C', 'B', 'B', 'B', 'B', 'A', 'A', 'B', 'B', 'A', 'B

In [21]:
pred

array(['B', 'B', 'B', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'B', 'A',
       'B', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'C', 'B', 'B', 'B', 'A',
       'A', 'B', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'B', 'A', 'B',
       'B', 'B', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'A', 'A', 'B', 'A',
       'B', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'B',
       'B', 'A', 'B', 'A', 'B', 'B', 'B', 'A', 'A', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'A', 'B', 'A', 'A', 'B', 'B', 'B', 'B', 'A',
       'A', 'B', 'B', 'A', 'B', 'A', 'A', 'B', 'B', 'A', 'B', 'B', 'A',
       'B', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'B',
       'B', 'B', 'A', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'A', 'B', 'A', 'B',
       'B', 'B', 'A', 'B', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'C', 'B', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'A', 'A', 'B', 'B', 'B', 'B

### xgboost

In [22]:
from xgboost import XGBClassifier
import xgboost as xgb

In [23]:
model_xgb = XGBClassifier()
model_xgb.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [24]:
pred_xgb = model_xgb.predict(x_test)

In [25]:
accuracy_score(y_test,pred_xgb)

0.7076923076923077

In [26]:
pred_xgb

array(['B', 'B', 'B', 'B', 'A', 'B', 'B', 'A', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'C', 'B', 'B', 'A', 'A',
       'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'A', 'A', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'A',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'B',
       'B', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'A', 'B', 'B', 'B', 'A', 'B', 'A', 'B', 'B', 'B

In [27]:
list(y_test).count('B')/len(y_test)

0.7282051282051282

In [28]:
len(y_test)

195

In [29]:
list(pred_xgb == y_test).count(False)

57

In [30]:
len(model_xgb.feature_importances_)

33

### RF

In [31]:
model_rf = RandomForestClassifier()
model_rf.fit(x_train, y_train)
pred_rf = model.predict(x_test)
accuracy_score(y_test,pred_rf)

D:\Program Files (x86)\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.6820512820512821

### 不使用onehot

In [ ]:
label_encoder = preprocessing.LabelEncoder()

In [ ]:
data_to_numeric = ['school','sex','address','famsize','Pstatus','Mjob','Fjob','reason','guardian','nursery','internet','famsup']

In [ ]:
data_demo3 = data[demographic_data_lst]

In [ ]:
for i in data_to_numeric:
    data_demo3[i]= label_encoder.fit_transform(data_demo3[i])

In [ ]:
data_demo3

In [ ]:
train_data_header = data_demo3.columns.tolist()
train_data = data_demo3.values
train_data.shape
train_x = train_data[:, 0:-3]
train_y = data.values[:, -1]

In [ ]:
from sklearn.model_selection import train_test_split
x_train1,x_test1,y_train1,y_test1 = train_test_split(train_x,train_y,test_size=0.3,random_state=123)

In [ ]:
model.fit(x_train1, y_train1)
pred = model.predict(x_test1)
accuracy_score(y_test1,pred)

In [ ]:
model_xgb = XGBClassifier()
model_xgb.fit(x_train1, y_train1)
pred_xgb = model_xgb.predict(x_test1)
accuracy_score(y_test1,pred_xgb)

### RF

In [ ]:
model_rf = RandomForestClassifier()
model_rf.fit(x_train1,y_train1)
pred_rf = model.predict(x_test1)
accuracy_score(y_test1,pred_rf)